In [1]:
from sklearn.datasets import fetch_openml
load_boston = fetch_openml(data_id=531, as_frame=True, parser='pandas')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [23]:
# Gather Data
boston_dataset = load_boston
data = pd.DataFrame(data=boston_dataset.data, dtype=np.float64, columns=boston_dataset.feature_names) #Force data types to float64
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = log_prices.values

In [36]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1,11))
# property_stats[0][CRIME_IDX] = features["CRIM"].mean()
# property_stats[0][ZN_IDX] = features["ZN"].mean()
# property_stats[0][CHAS_IDX] = features["CHAS"].mean()

property_stats = features.mean().values.reshape(1, 11)

In [26]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [50]:
def get_log_estimate(nr_rooms, 
                     students_per_classroom, 
                     next_to_river=False, 
                     high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # Make prediction
    log_estimate = regr.predict(property_stats)[0]
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
        
    return log_estimate, upper_bound, lower_bound, interval

In [52]:
get_log_estimate(3, 20, 1, 0)

C:\Users\filip\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.7767581914803996, 2.96427032667753, 2.5892460562832693, 68)

In [53]:
np.median(boston_dataset.target)

21.2

In [56]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE/np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(nr_rooms=9, students_per_classroom=15, 
                                               next_to_river=False, high_confidence=False)

# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f"The estimated property value is {rounded_est}.")
print(f"At {conf}% confidence the valuation range is")
print(f"USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.")

The estimated property value is 827000.0.
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


C:\Users\filip\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [71]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True is the property is next to the river, False otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval
    
    """
    
    if rm < 1 or ptratio < 1:
        print("That is unrealistic. Try again.")
        return
    
    log_est, upper, lower, conf = get_log_estimate(nr_rooms=rm, students_per_classroom=ptratio, 
                                               next_to_river=chas, high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f"The estimated property value is {rounded_est}.")
    print(f"At {conf}% confidence the valuation range is")
    print(f"USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.")

In [69]:
get_dollar_estimate(rm=2, ptratio=0, chas=True)

That is unrealistic. Try again.


In [72]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.


C:\Users\filip\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
